# Segmentation debugging

In [37]:
import sys
sys.path.append('/home/g/Desktop/git/nodules/src/')
sys.path.append('/home/g/Desktop/git/nodules/keras/')
sys.path.append('/home/g/Desktop/git/nodules/models/')
sys.path.append('/home/g/Desktop/git/keras/keras/preprocessing/')

import os
import copy
import time
#from glob import glob
import numpy as np
import pandas as pd
import SimpleITK as sitk
#import matplotlib.pyplot as plt

import seg3d 
import candidates as cd
import candgen as cg
import segment as sgmt

#from skimage import feature
from skimage import measure
import scipy.ndimage
#import skimage

In [9]:
DATAPATH =  '/scr/nodules/luna/val/subset0/'
ANNOTATIONS = '/home/g/Desktop/git/nodules/resources/annotations.csv'
WEIGHTSPATH = '/home/g/Desktop/lowloss_dsb_weights_062017.h5'
OUTFILE = '/home/g/Desktop/assess_nn_dbg.txt'
STRIDE = 32

In [4]:
filelist, filepathlist = sgmt.getFilelist(DATAPATH)
model = seg3d.getModel()
model.load_weights(WEIGHTSPATH)
candlist = pd.read_csv(ANNOTATIONS)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 64, 64, 64) 0                                            
____________________________________________________________________________________________________
conv3d_1 (Conv3D)                (None, 128, 62, 62, 6 3584        input_1[0][0]                    
____________________________________________________________________________________________________
p_re_lu_1 (PReLU)                (None, 128, 62, 62, 6 30505984    conv3d_1[0][0]                   
____________________________________________________________________________________________________
conv3d_2 (Conv3D)                (None, 128, 60, 60, 6 442496      p_re_lu_1[0][0]                  
___________________________________________________________________________________________

In [10]:
n=2
currentNodules = candlist[candlist['seriesuid'] == filelist[n]]
itk = sitk.ReadImage(filepathlist[n])
image, _ = cg.loadItk(filepathlist[n],None)
origin = np.array(itk.GetOrigin())
spacing = np.array(itk.GetSpacing())
coords = sgmt.getCoords(currentNodules, origin, spacing)

segShape = sgmt.getSegmentationShape(image.shape)
lpad, hpad = sgmt.getPadding(segShape,image)
segImage = np.zeros(segShape)
segImage[lpad[0]:segShape[0]-hpad[0],lpad[1]:segShape[1]-hpad[1],lpad[2]:segShape[2]-hpad[2]] = image 
segImage = np.clip(segImage,-1000,1000)
segImage += 1000
segImage *= (255./2000.)

stride = STRIDE
strides = copy.deepcopy(segShape)
strides /= stride
#print segShape
#print strides
segmentation = sgmt.segmentImage(segImage, model, strides, stride)
lpad -= 16 
hpad -= 16
pseg = segmentation[lpad[0]:segmentation.shape[0]-hpad[0],lpad[1]:segmentation.shape[1]-hpad[1],lpad[2]:segmentation.shape[2]-hpad[2]]
seg = np.zeros(pseg.shape)
seg = np.where(pseg>1e-2,1,0)

SEGMENTING ...
ASSESSING ...


NameError: name 'measure' is not defined

In [45]:
labels = measure.label(seg, connectivity=1)
props = measure.regionprops(labels, labels)#, pseg)
propsi = measure.regionprops(labels, image)
#assess(OUTFILE,labels,coords,props,filelist[n], propsi=propsi)

In [41]:
label = np.zeros(labels.shape).astype(int)

for coord in coords:
  time1 = time.time()
  c3,c2,c1 = coord
  l = labels[c1,c2,c3]
  #l += 1
  if l != 0:
    #time1 = time.time()
    label *= 0
    #label = np.zeros(labels.shape).astype(int)
    label = np.where(labels==l,1,0)
    prop_prob = measure.regionprops(label,pseg)
    #com = scipy.ndimage.measurements.center_of_mass(labels)
    print (c1,c2,c3)
    print prop_prob[0].centroid
    #print com
    #print np.sum(label)
    print prop_prob[0].area
  time2 = time.time()
  print str(time2-time1)+'s'

(168, 270, 243)
(168.13443396226415, 270.81620619946091, 242.08018867924528)
5936
4.11047005653s
(162, 221, 97)
(162.26272912423624, 221.04073319755599, 96.957230142566189)
491
6.30972981453s
(183, 157, 74)
(184.22095959595958, 156.96969696969697, 73.931818181818187)
792
6.19247794151s


In [19]:
props[0].label

1

In [24]:
len(labels)

368

In [46]:
len(props)

1806

In [47]:
len(propsi)

1806

In [16]:
print currentNodules

                                              seriesuid      coordX  \
1011  1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803...   60.775061   
1012  1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803...  -85.421039   
1013  1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803... -108.527516   

         coordY      coordZ  diameter_mm  
1011  74.123970 -214.782347    25.233202  
1012  25.261721 -220.833830     9.756615  
1013 -38.825247 -199.207307    10.989140  
